Data cleaning, analysis and visualization in Python using Jupyter Notebooks.

Importing the necessary libraries.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


Acquiring the data from the CSV file.

In [ ]:
netflix = pd.read_csv('netflix_titles.csv')

The data at first glance

In [ ]:
#First 5 rows of the dataframe
netflix.head()

In [ ]:
#Number of rows and columns
netflix.shape

In [ ]:
#Statistical summary of our data
netflix.describe()

In [ ]:
netflix.info()

Now that our data has been imported correctly and we've taken a look at it, we can start noticing a few problems which may skew our analysis process:

1.Null values are present in a few columns. Let's get the percentage of null values for each column:

In [ ]:
percentages = netflix.isnull().mean()*100
null_columns = round(percentages[percentages > 0], 2)
null_columns.sort_values(ascending=False, inplace=True)
null_columns

We can clearly tell there will be a problem when performing our analysis since more than a quarted of the "director" column is filled with null values, and almost 10% of the "country" and "cast" columns are null as well.

To fix this, we will fill these columns with an empty string value (""), since this will not affect our charts and plots.

However, we can see a small percentage of missing values in the "date_added", "duration" and "rating" columns as well. Since these will make a large part of our plots, we shall remove the rows that containg null values in these columns in order to not skew with our analysis.

In [ ]:
#Filling in the null values
netflix[['director', 'country', 'cast']].dtypes

In [ ]:
netflix[['director', 'country', 'cast']] = netflix[['director', 'country', 'cast']].fillna("Unknown")
netflix.head()

Now that we have filled in the missing values for these columns, we need to do something about the rest. Since filling them as well would skew our plots and there is a very small amount of data missing, we will drop the rows that contain missing data in the "date_added", "duration" and "rating" columns.

In [ ]:
netflix.isnull().sum()

Since these are the only columns that still have null values, we can drop the rows as follows:

In [ ]:
netflix.dropna(inplace=True)
netflix.info()

In [ ]:
netflix.isnull().any()

Now that our data is free of null values, we should start cleaning the not-null values.

In [ ]:
netflix.head()

Removing the trailing whitespaces from all of the string and mixed columns.

In [ ]:
for i in netflix.columns:
    if netflix[i].dtype=='O':
        netflix[i] = netflix[i].str.strip()
netflix.head()

In [ ]:
netflix['date_added'].dtype

Taking a look at the "date_added" column, we can see it is of type 'object' meaning it's a string. We will separate the column into 2 distinct columns: one for the year and one for the month.

In [ ]:
from datetime import datetime
years = []
months = []
for i in range(len(netflix)):
    date = netflix['date_added'].iloc[i]
    date = datetime.strptime(date, "%B %d, %Y")

    years.append(date.year)
    months.append(date.month)

if 'year_added' not in netflix.columns and 'month_added' not in netflix.columns:
    netflix.insert(7, 'year_added', years, allow_duplicates=True)
    netflix.insert(8, 'month_added', months, allow_duplicates=True)
netflix.head()

Now that our "date_added" columns has been cleaned, we need to take a look at the other columns to see which ones containt corrupted values. We'll start with the "type" column.


In [ ]:
netflix.type.value_counts()

We can see the column is formatted correctly as there can only be Movies and TV Shows as the values. We will later separate the dataframe into 2 subsets, one for each type, but for now, we have to finish the cleaning process.

Now we will have a look at the rating column


In [ ]:
netflix.rating.value_counts()

All of these are viable values for a "rating" column. This means we are almost finised with the data cleaning. All that is left to do is to examine the "duration" column and partition the dataset accordingly into movies and TV shows. 

In [ ]:
(netflix['duration'].str.contains('min') == (netflix['type'] == 'Movie'))

Our hypothesis is that every entry in the dataset that is listed as a movie in the "type" column will have a duration listed in the number of minutes. We will test if this is true for every entry in the dataset down below.

In [ ]:
(netflix['duration'].str.contains('min') == (netflix['type'] == 'Movie')).all()


It seems we were correct, we will now do the same thing for the TV shows, testing if every TV show's meassure unit in the "duration" column is the number of seasons.

In [ ]:
(netflix['duration'].str.contains('Season') == (netflix['type'] == 'TV Show')).all()


Perfect, this means that splitting our dataset into movies and TV shows is as simple as this.

In [ ]:
movies = netflix.groupby('type').get_group('Movie')
movies

In [ ]:
shows = netflix.groupby('type').get_group('TV Show')
shows

To test if the records have been grouped correctly, we can do the following:

In [ ]:
movies['type'].value_counts()

In [ ]:
shows['type'].value_counts()

In [ ]:
len(netflix) == len(shows) + len(movies)

Ok, we have our records split into movies and TV shows correctly, now to clean up the "duration" column accordingly.

In [ ]:
if 'duration' in movies.columns:
    duration_list = movies['duration'].str.split()
    duration_list = [i[0] for i in duration_list]
    movies.iloc[0:, 11] = duration_list
movies = movies.rename(columns = {'duration': 'length'})
movies



And now we will do the same for our TV shows.


In [ ]:
if 'duration' in shows.columns:
    duration_list = shows['duration'].str.split()
    duration_list = [i[0] for i in duration_list]
    shows.iloc[0:, 11] = duration_list
shows = shows.rename(columns = {'duration': 'number_of_seasons'})
shows

Perfect! Our data is sparkling clean and now it's time for the analysis to begin. In order to get a better understanding of the data we have to work with, we will first plot how Netflix has evolved in terms of content over the last decade.

In [ ]:
fig = plt.figure(figsize=(13, 7))
releases = netflix[netflix.release_year >= 2011].release_year.value_counts().sort_index()
ax = fig.add_axes([0,0,1,1])
ax.bar(np.arange(2011, 2022), releases, width=0.5)
ax.set_xticks(np.arange(2011, 2022))
ax.set_xlabel('Year')
ax.set_ylabel('Number of releases')
plt.show()


Let us evaluate our findings. From the graphs we can tell Netflix has been steadily growing its library of both Movies and TV shows, reaching over 6x as many titles in 2018 compared to 2011, and this number has remained almot constant through 2019 and 2020. 

Presumably, due to the pandemic, the number of titles released in 2021 is significantly lower than the last few years, however.

And now let us compare the number of released movies vs TV shows in the last decade.

In [ ]:
fig = plt.figure(figsize=(13, 7))
movie_releases = movies[movies.release_year >= 2011].release_year.value_counts().sort_index()
show_releases = shows[shows.release_year >= 2011].release_year.value_counts().sort_index()
ax = fig.add_axes([0,0,1,1])
ax.bar(np.arange(2011, 2022), movie_releases, width=0.5, color='#00b894', label='Movies')
ax.bar(np.arange(2011, 2022), show_releases, width=0.5, color='#e17055', bottom = movie_releases, label='TV Shows')
ax.set_xticks(np.arange(2011, 2022))
ax.legend()
ax.set_xlabel('Year')
ax.set_ylabel('Number of releases')
plt.show()

Interesting, it seems like the proportion of TV shows compared to movies has significantly grown over the last decade, reaching almost a perfect balance between the 2.

In fact, let's visualize the proportion of TV Shows compared to movies. 

In [ ]:
fig, (a1, a2) = plt.subplots(1,2, figsize=(13, 7))
a1.pie([len(movies), len(shows)], labels=['Movies', 'TV Shows'], autopct='%1.1f%%')
a1.set_title('Total number of TV shows vs movies of all time')
a2.pie([len(movies[movies.release_year == 2021]), len(shows[shows.release_year == 2021])], labels=['Movies', 'TV Shows'], colors=['#e17055', '#00b894' ] , autopct='%1.1f%%')
a2.set_title('Total number of TV shows vs movies released in 2021')
plt.show()

While the number of movies is still overall larger than the number of TV shows, we can see a trend towards more TV shows being added to the platform instead of movies.

Now it's about time we took a look at the "country" column to see what are the most common places for movie and TV show production.

In [ ]:
netflix['country']

We can see there are titles which have been produced in multiple countries, thus, we need to separate them into the corresponding countries.

In [ ]:
def get_country_values(df):
    country_values = pd.Series(dtype=int)
    for c_list in df.country:
        c_list = c_list.split(",")
        for c in c_list:
            c = c.strip()
            if c != 'Unknown':
                if c in country_values.index:
                    country_values[c] +=1
                else:
                    country_values[c] = 1
    return country_values

In [ ]:
print(get_country_values(netflix))

Perfect, now we know the exact amount of movies and shows produced in each country, we will move on to visualizing this data.

In [ ]:
total_countries = get_country_values(netflix)
total_countries.sort_values(ascending=False, inplace=True)
top_10_countries = total_countries[:10]

fig, ax = plt.subplots(figsize=(13, 7))
ax.bar(top_10_countries.index, top_10_countries, width = 0.5, color=(0.2, 0.4, 0.6, 0.6))
ax.set_xlabel('Country')
ax.set_ylabel('Number of titles')
plt.show()

And let's see what are the countries which produce the most movies vs TV shows.

In [ ]:
movie_countries = get_country_values(movies).sort_values(ascending = False)[:10]
show_countries = get_country_values(shows).sort_values(ascending = False)[:10]

fig, (a1, a2) = plt.subplots(1, 2, figsize = (21, 11))
a1.barh(movie_countries.index[::-1], movie_countries[::-1], color = '#e17055')
a1.set_xlabel('Number of movies produced')
a1.set_ylabel('Country')
a2.barh(show_countries.index[::-1], show_countries[::-1], color = '#00b894')
a2.set_xlabel('Number of TV shows produced')
a2.set_ylabel('Country')
plt.show()

To end the analysis right, we shall visualize the correlation between the countries where movies and TV shows have been produced and their respective ratings.

In [ ]:
plt.figure(figsize=(13, 7))
top_countries = get_country_values(netflix).sort_values(ascending = False).index[:5]
g = sns.histplot(x = 'rating', y='country', data=netflix[netflix["country"].isin(top_countries)])
g.set_xlabel("Ratings")
g.set_ylabel("Countries")
g.set_title("Top 5 ratings per country")
plt.show()